-------------------------------------------
Chains in LangChain
Outline-->

1.LLMChain
2.Sequential Chains
3.SimpleSequentialChain
4.SequentialChain
5.Router Chain
===================

In [1]:
import os
import warnings
warnings.filterwarnings('ignore')
from dotenv import load_dotenv, find_dotenv

load_dotenv(dotenv_path=r"C:\Users\hasan\Rafi_SAA\python_practices\LLM_Langchain\llm_openai\.env")


True

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

---------------------------
Simple LLM chain
===================

In [ ]:
chat = ChatOpenAI(
    model="openai/gpt-4o",
    openai_api_key=os.getenv("GPT_github_access_token"),
    openai_api_base="https://models.github.ai/inference"

)

In [6]:
prompt = ChatPromptTemplate.from_template("What is a good name for a company that makes {product}?")

In [ ]:
chain = LLMChain(llm=chat, prompt=prompt)

In [ ]:
product  = "sports shoes"
chain.run(product)

In [30]:
skin_tone = "light"
body_type = "skinny"
available_clothes = "t-shirts, jeans, sneakers, jackets, hats, scarves, belts, sunglasses, watches, gown, ties, blazers, skirts"
available_clothes = "".join(available_clothes)


In [31]:
prompt_fashion = ChatPromptTemplate.from_template("You are a fashion expert. Suggest a type of clothing based on the :{available_clothes}, my {skin_tone} skin tone , {body_type} body type.")


In [33]:
chain2 = LLMChain(llm= chat, prompt= prompt_fashion)
result2 = chain2.run({
    "available_clothes": available_clothes,
    "skin_tone": skin_tone,
    "body_type": body_type
})
print(result2)


Certainly! Based on your light skin tone and skinny body type, here are some clothing suggestions to flatter your frame and complement your complexion:

### **Casual Look**
1. **T-Shirts**: Opt for light or pastel-colored t-shirts (like baby blue, blush pink, or light yellow) to complement your skin tone. Slim-fit or tailored styles work well to enhance your frame.
2. **Jeans**: Skinny or slim-fit jeans in medium-wash blue or black will elongate your legs and create a sleek silhouette.
3. **Sneakers**: Go for classic white sneakers or neutral tones to maintain a clean and stylish appearance.
4. **Jackets**: Lightweight bomber jackets or tailored denim jackets in earthy tones or navy blue will add structure without overwhelming your frame.
5. **Hats**: A fedora or a classic beanie in neutral tones like beige, gray, or camel would suit your light skin tone beautifully.

### **Chic Winter Look**
1. **Scarves**: Try chunky knit scarves in warm tones such as burgundy, mustard, or forest gre

In [ ]:
print(result2)

-------------------------------------
simple sequential Chain
=========================

In [11]:
from langchain.chains import SimpleSequentialChain
# Simple LLM chain

In [23]:
body_type = "athletic"
available_clothes = "t-shirts, jeans, sneakers, jackets, hats, scarves, belts, sunglasses, watches, gown, ties, blazers, skirts"
available_clothes = "".join(available_clothes)
print(available_clothes)

t-shirts, jeans, sneakers, jackets, hats, scarves, belts, sunglasses, watches, gown, ties, blazers, skirts


In [24]:
# first prompt template
prompt1 = ChatPromptTemplate.from_template("You are a fashion expert. Suggest a type of clothing based on the body type: {body_type}")

#chain1
chain1 = LLMChain(llm= chat, prompt = prompt1, output_key = "clothing_suggestion")

In [25]:

prompt2 = ChatPromptTemplate.from_template(
    "You are a fashion expert. Based on your suggestion and my {available_clothes}, make a suggestion. "
    "If no clothes are available, then suggest from your suggestion."
)

chain2 = LLMChain(llm= chat, prompt = prompt2)


In [ ]:
# prompt2.format(available_clothes=available_clothes)
# print(available_clothes)

t-shirts, jeans, sneakers, jackets, hats, scarves, belts, sunglasses, watches, gown, ties, blazers, skirts


In [26]:
overall_chain = SimpleSequentialChain(chains = [chain1, chain2], verbose = True)

In [28]:
result  = overall_chain.run(body_type)
print(result)



> Entering new SimpleSequentialChain chain...
For an athletic body type, the goal is often to create the illusion of curves and balance your naturally toned and straight silhouette. Here are some clothing suggestions that work well for athletic builds:

### Tops:
- **Peplum Tops:** Add volume to the waist and create a curvier appearance.
- **Wrap Tops:** Accentuate the waist and add a feminine shape.
- **Off-the-Shoulder Tops:** Highlight your shoulders and create softness.
- **Ruffled or Draped Details:** Add volume to your upper body.

### Bottoms:
- **High-Waisted Jeans or Pants:** Define your waist and create curves.
- **Wide-Leg Trousers or Palazzo Pants:** Balance a straight silhouette with flowy, voluminous bottoms.
- **A-Line Skirts:** Give the illusion of hips and add femininity.
- **Paper Bag-Waist Pants:** Add emphasis to your waist and create shape.

### Dresses:
- **Fit-and-Flare Dresses:** Highlight your toned upper body and add volume to your lower half.
- **Bodycon Dr